# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

## Observation of results:
    1. Upon using the current method for locating cities, more cities are found in northern hemisphere than southern hemisphere. The latitude coverage in northern hemisphere is up to 80°, however, only up to -60° range was found for southern hemisphere. This is due to less proportion of ocean coverage in northern hemisphere.
    2. By fitting linear models on temperature vs. latitude, one could note that the models give moderate to good R-square value. It shows an increase of temperature with the decrease of latitudes.
    3. By doing similar analysis as in 2, cloudiness, humidity and wind speed do not show a strong linear relationship with temperature.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
temperature = []
humidity = []
cloudiness = []
wind_speed = []
longitudes = []
latitudes = []
city_list = []
country = []
count = 1
for city in cities:
    params = {
        'appid': weather_api_key,
        'q':city,
        'units': 'imperial'
    }
    url = f'http://api.openweathermap.org/data/2.5/weather'
    response = requests.get(url, params=params).json()

    try:
        temperature.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_speed.append(response['wind']['speed'])
        longitudes.append(response['coord']['lon'])
        latitudes.append(response['coord']['lat'])        
        print(f'Processing record {count}: {city}')
        city_list.append(city)
        country.append(response['sys']['country'])
        count+=1
        
    except KeyError:
        print(f'City:{city} is not found. Skipping... ')
        
print(f'Data retrival complete!')

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_df = pd.DataFrame({
    'City_Name': city_list,
    'country':country,
    'Latitude': latitudes,
    'Longitude': longitudes,
    'Temperature/F': temperature,
    'Humidity/%': humidity,
    'Cloudiness/%':cloudiness,
    'Wind_Speed/mph':wind_speed
})
weather_df.to_csv(output_data_file, index=False)
weather_df.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

In [ ]:
# Making scatter plots

plt.figure(figsize=(15,15))

plt.subplot(221)
plt.scatter(weather_df.Latitude, weather_df['Temperature/F'], alpha=0.5)
plt.title('Temperature (F) vs. Latitude', size=12)
plt.xlabel('Latitude (°)', size=12)
plt.ylabel('Temperature (°F)', size=12)

plt.subplot(222)
plt.scatter(weather_df.Latitude, weather_df['Humidity/%'], alpha=0.5)
plt.title('Humidity (%) vs. Latitude', size=12)
plt.xlabel('Latitude (°)', size=12)
plt.ylabel('Humidity (%)', size=12)

plt.subplot(223)
plt.scatter(weather_df.Latitude, weather_df['Cloudiness/%'], alpha=0.5)
plt.title('Cloudiness (%) vs. Latitude', size=12)
plt.xlabel('Latitude (°)', size=12)
plt.ylabel('Cloudiness (%)', size=12)

plt.subplot(224)
plt.scatter(weather_df.Latitude, weather_df['Wind_Speed/mph'], alpha=0.5)
plt.title('Wind Speed (mph) vs. Latitude', size=12)
plt.xlabel('Latitude (°)', size=12)
plt.ylabel('Wind Speed (mph)', size=12)
plt.savefig("output_data/global.png")
plt.show()

## Linear Regression

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
northern_df = weather_df.loc[weather_df.Latitude >= 0]
southern_df = weather_df.loc[weather_df.Latitude < 0]

In [ ]:
def linear_regression(x, y):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
    reg_value = slope*x + intercept
    
    return (slope, intercept, reg_value, rvalue)

In [ ]:
df_list = [northern_df, southern_df]
column_list = ['Temperature/F','Humidity/%', 'Cloudiness/%','Wind_Speed/mph']
label_list = ['Northern Hemisphere', 'Southern Hemisphere']

plt.figure(figsize=(12,10))
for i in range(len(label_list)):
    for j in range(len(column_list)):
        
# Performing linear regression.
        result = linear_regression(df_list[i].Latitude, df_list[i][column_list[j]])        
# Creating subplots        
        plt.subplot(2,2,j+1)
# Plotting real data vs fitted data:
        plt.scatter(df_list[i].Latitude, df_list[i][column_list[j]], alpha=0.5, label=label_list[i])
        plt.plot(df_list[i].Latitude, result[2], 'r--', alpha=0.5)
        
        plt.title(f'{column_list[j]} (F) vs. Latitude', size=12)
        plt.xlabel('Latitude (°)', size=12)
        plt.ylabel(f'{column_list[j]}', size=12)
        
# Display subplot#3 legend at a better location    
        if j == 2:
            plt.ylim(-5,120)
        plt.legend(loc='best')
        print(f'{label_list[i]} {column_list[j]}:R Square={result[3]**2}')
plt.tight_layout()
plt.savefig("output_data/Northern-Southern.png")
plt.show()